# MNIST with Virtual Branching

In [1]:
import tensorflow as tf
import numpy as np
import os
from scipy.special import softmax
import matplotlib.pyplot as plt
import time
from sklearn.manifold import TSNE

In [2]:
import vbranch as vb

In [3]:
save = False
model_id = 1
architecture = 'cnn'

## Load Data

In [4]:
input_dim = 784
num_classes = 10

In [5]:
(X_train, y_train_one_hot), (X_test, y_test_one_hot) = vb.datasets.mnist.load_data(format=architecture)

## Train

### Build Model

In [6]:
BATCH_SIZE = 32
EPOCHS = 10
STEPS_PER_EPOCH = 100
NUM_BRANCHES = 3
SHARED_FRAC = 0
model_path = os.path.join('models', 'vb-mnist-{}-B{:d}-S{:.2f}_{:d}'.format(architecture,
    NUM_BRANCHES, SHARED_FRAC, model_id))

In [7]:
model_path

'models/vb-mnist-cnn-B3-S0.00_1'

In [8]:
tf.reset_default_graph()

train_data = (X_train.astype('float32'), y_train_one_hot)
test_data = (X_test.astype('float32'), y_test_one_hot)

batch_size = tf.placeholder('int64', name='batch_size')

train_datasets = []
test_datasets = []
inputs = [None] * NUM_BRANCHES
labels_one_hot = [None] * NUM_BRANCHES
train_init_ops = []
test_init_ops = []

for i in range(NUM_BRANCHES):
    train_datasets.append(tf.data.Dataset.from_tensor_slices(train_data).\
        batch(batch_size).repeat().\
        shuffle(buffer_size=4*BATCH_SIZE))

    test_datasets.append(tf.data.Dataset.from_tensor_slices(test_data).\
        batch(batch_size))
    
    iterator = tf.data.Iterator.from_structure(train_datasets[i].output_types, 
                                           train_datasets[i].output_shapes)
    inputs[i], labels_one_hot[i] = iterator.get_next(name='input_'+str(i+1))    

    train_init_ops.append(iterator.make_initializer(train_datasets[i]))
    test_init_ops.append(iterator.make_initializer(test_datasets[i], 
                                                name='test_init_op_'+str(i+1)))

In [9]:
if architecture == 'fcn':
    model = vb.vbranch_simple_fcn(inputs,
        ([128]*NUM_BRANCHES, int(128*SHARED_FRAC)), ([10]*NUM_BRANCHES, int(10*SHARED_FRAC)),
        branches=NUM_BRANCHES, name='model_' + str(model_id))
elif architecture == 'cnn':
    model = vb.vbranch_simple_cnn(inputs, (num_classes, 0),
        ([16]*NUM_BRANCHES, int(16*SHARED_FRAC)), ([32]*NUM_BRANCHES, int(32*SHARED_FRAC)),
        branches=NUM_BRANCHES, name='model_' + str(model_id))

In [10]:
model.summary()

i   Layer name         Output shape      Num param  Inbound            
-----------------------------------------------------------------------
    Input              [None,28,28,1]                                  
-----------------------------------------------------------------------
    Input              [None,28,28,1]                                  
-----------------------------------------------------------------------
    Input              [None,28,28,1]                                  
-----------------------------------------------------------------------
0   conv2d_1_1         [None,26,26,16]   480        input              
                       [None,26,26,16]                                 
                       [None,26,26,16]                                 
-----------------------------------------------------------------------
1   bn_1_1             [None,26,26,16]   96         conv2d_1_1         
                       [None,26,26,16]                          

In [11]:
tf.global_variables()

[<tf.Variable 'model_1/conv2d_1_1_vb1_f:0' shape=(3, 3, 1, 16) dtype=float32_ref>,
 <tf.Variable 'model_1/conv2d_1_1_vb1_b:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'model_1/conv2d_1_1_vb2_f:0' shape=(3, 3, 1, 16) dtype=float32_ref>,
 <tf.Variable 'model_1/conv2d_1_1_vb2_b:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'model_1/conv2d_1_1_vb3_f:0' shape=(3, 3, 1, 16) dtype=float32_ref>,
 <tf.Variable 'model_1/conv2d_1_1_vb3_b:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'model_1/bn_1_1_vb1_unique_to_unique_scale:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'model_1/bn_1_1_vb1_unique_to_unique_beta:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'model_1/bn_1_1_vb2_unique_to_unique_scale:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'model_1/bn_1_1_vb2_unique_to_unique_beta:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'model_1/bn_1_1_vb3_unique_to_unique_scale:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'model_1/bn_1_1_vb3_unique_to_unique_beta:0' shape=(16

### Declare Training Ops

In [13]:
def get_shared_unshared_vars():
    """
    Get shared variables (in order to later average gradients)
    and unshared variables (unique to each branch)"""
    
    shared_vars = []
    unshared_vars = [[] for i in range(NUM_BRANCHES)]
    
    for var in tf.global_variables():
        if 'shared_to_shared' in var.name:
            shared_vars.append(var)
        else:
            for i in range(NUM_BRANCHES):
                if 'vb'+str(i+1) in var.name:
                    unshared_vars[i].append(var)
            
    return shared_vars, unshared_vars

In [14]:
shared_vars, unshared_vars = get_shared_unshared_vars()

In [15]:
unshared_vars

[[<tf.Variable 'model_1/conv2d_1_1_vb1_f:0' shape=(3, 3, 1, 16) dtype=float32_ref>,
  <tf.Variable 'model_1/conv2d_1_1_vb1_b:0' shape=(16,) dtype=float32_ref>,
  <tf.Variable 'model_1/bn_1_1_vb1_unique_to_unique_scale:0' shape=(16,) dtype=float32_ref>,
  <tf.Variable 'model_1/bn_1_1_vb1_unique_to_unique_beta:0' shape=(16,) dtype=float32_ref>,
  <tf.Variable 'model_1/conv2d_1_2_vb1_f:0' shape=(3, 3, 16, 16) dtype=float32_ref>,
  <tf.Variable 'model_1/conv2d_1_2_vb1_b:0' shape=(16,) dtype=float32_ref>,
  <tf.Variable 'model_1/bn_1_2_vb1_unique_to_unique_scale:0' shape=(16,) dtype=float32_ref>,
  <tf.Variable 'model_1/bn_1_2_vb1_unique_to_unique_beta:0' shape=(16,) dtype=float32_ref>,
  <tf.Variable 'model_1/conv2d_2_1_vb1_f:0' shape=(3, 3, 16, 32) dtype=float32_ref>,
  <tf.Variable 'model_1/conv2d_2_1_vb1_b:0' shape=(32,) dtype=float32_ref>,
  <tf.Variable 'model_1/bn_2_1_vb1_unique_to_unique_scale:0' shape=(32,) dtype=float32_ref>,
  <tf.Variable 'model_1/bn_2_1_vb1_unique_to_unique_bet

In [16]:
# Multi output loss
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)

losses = []
# Store gradients from shared variables over each branch
shared_grads = []
unshared_train_ops = []

for i in range(NUM_BRANCHES):
    loss = vb.losses.softmax_cross_entropy_with_logits(labels=labels_one_hot[i], 
                                                       logits=model.output[i], 
                                                       name='loss_'+str(i+1))
    losses.append(loss)
    
    # Compute gradients of shared vars for each branch (but don't apply)
    if len(shared_vars) > 0:
        shared_grads.append(optimizer.compute_gradients(loss, var_list=shared_vars))
        
    # Apply gradients for unshared vars for each branch
    if len(unshared_vars[i]) > 0:
        unshared_train_ops.append(optimizer.minimize(loss, var_list=unshared_vars[i]))

In [17]:
# Take average of the gradients over each branch
mean_shared_grads = []

for v, var in enumerate(shared_vars):
    grad = tf.reduce_mean([shared_grads[i][v][0] for i in range(NUM_BRANCHES)], [0])
    mean_shared_grads.append((grad, var))
    
if len(shared_vars) > 0:
    shared_train_op = optimizer.apply_gradients(mean_shared_grads)
else:
    shared_train_op = []

In [18]:
mean_shared_grads

[]

In [19]:
train_ops = [unshared_train_ops, shared_train_op]

In [20]:
# Train accuracies
train_acc_ops = []
for i in range(NUM_BRANCHES):
    pred_max = tf.one_hot(tf.argmax(tf.nn.softmax(model.output[i]), axis=-1), 
                          num_classes)
    train_acc_op = tf.reduce_mean(tf.reduce_sum(labels_one_hot[i]*pred_max, [1]), 
                                  name='train_acc_'+str(i+1))
    train_acc_ops.append(train_acc_op)

# Test accuracy
pred = tf.nn.softmax(tf.reduce_mean(model.output.to_list(), [0]))
pred_max = tf.one_hot(tf.argmax(pred, axis=-1), num_classes)
test_acc_op = tf.reduce_mean(tf.reduce_sum(labels_one_hot[0]*pred_max, [1]), 
                             name='test_acc')

### Run Ops

In [21]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for e in range(EPOCHS):
        print("Epoch {}/{}".format(e + 1, EPOCHS))
        progbar = tf.keras.utils.Progbar(STEPS_PER_EPOCH)
        
        sess.run(train_init_ops, feed_dict={batch_size: BATCH_SIZE})

        for i in range(STEPS_PER_EPOCH):
            _, train_losses, train_accs = sess.run([train_ops, losses, 
                                                 train_acc_ops])
            
            prog_vals = [('loss_'+str(b+1),train_losses[b]) for b in range(NUM_BRANCHES)]
            prog_vals += [('acc_'+str(b+1),train_accs[b]) for b in range(NUM_BRANCHES)]
            
            if i == STEPS_PER_EPOCH - 1:
                sess.run(test_init_ops, feed_dict={batch_size: len(X_test)})
                val_losses, val_acc, indiv_accs = sess.run([losses, test_acc_op, 
                                                            train_acc_ops])
                
                prog_vals += [("val_loss", np.mean(val_losses)), ("val_acc", val_acc)] + \
                    [('ind_acc_'+str(b+1), indiv_accs[b]) for b in range(NUM_BRANCHES)]
            
            progbar.update(i+1, values=prog_vals)
    
    if save:
        saver = tf.train.Saver()
        path = os.path.join(model_path, 'ckpt')
        saver.save(sess, path)

Epoch 1/10
100/100 [==============================] - 68s 678ms/step - loss_1: 1.2363 - loss_2: 1.3046 - loss_3: 1.3643 - acc_1: 0.6669 - acc_2: 0.6300 - acc_3: 0.6128 - val_loss: 0.6161 - val_acc: 0.9298 - ind_acc_1: 0.8957 - ind_acc_2: 0.8865 - ind_acc_3: 0.8890
Epoch 2/10
100/100 [==============================] - 41s 408ms/step - loss_1: 0.4608 - loss_2: 0.5048 - loss_3: 0.5309 - acc_1: 0.9044 - acc_2: 0.8922 - acc_3: 0.8869 - val_loss: 0.3167 - val_acc: 0.9546 - ind_acc_1: 0.9374 - ind_acc_2: 0.9312 - ind_acc_3: 0.9357
Epoch 3/10
100/100 [==============================] - 48s 477ms/step - loss_1: 0.2754 - loss_2: 0.3313 - loss_3: 0.3255 - acc_1: 0.9387 - acc_2: 0.9222 - acc_3: 0.9228 - val_loss: 0.2340 - val_acc: 0.9630 - ind_acc_1: 0.9453 - ind_acc_2: 0.9472 - ind_acc_3: 0.9442
Epoch 4/10
100/100 [==============================] - 39s 390ms/step - loss_1: 0.2153 - loss_2: 0.2414 - loss_3: 0.2248 - acc_1: 0.9484 - acc_2: 0.9413 - acc_3: 0.9450 - val_loss: 0.1748 - val_acc: 0.9694 

## Load Model

In [ ]:
test_init_ops = ['test_init_op_'+str(i+1) for i in range(NUM_BRANCHES)]
losses = ['loss_'+str(i+1)+':0' for i in range(NUM_BRANCHES)]
train_acc_ops = ['train_acc_'+str(i+1)+':0' for i in range(NUM_BRANCHES)]

inputs = ['input_{}:0'.format(i+1) for i in range(NUM_BRANCHES)]
labels_one_hot = ['input_{}:1'.format(i+1) for i in range(NUM_BRANCHES)]
outputs = ['model_{}/output_vb{}:0'.format(model_id, i+1) for i in range(NUM_BRANCHES)]

In [ ]:
with tf.Session() as sess:
    model_path = os.path.join('models', 'vb-mnist-{}-B{:d}-S{:.2f}_{:d}'.format(architecture,
        NUM_BRANCHES, SHARED_FRAC, model_id))
    meta_path = os.path.join(model_path, 'ckpt.meta')
    ckpt = tf.train.get_checkpoint_state(model_path)

    imported_graph = tf.train.import_meta_graph(meta_path)
    imported_graph.restore(sess, ckpt.model_checkpoint_path)

    sess.run(test_init_ops, feed_dict={'batch_size:0': len(X_test)})
    val_losses, val_acc, indiv_accs = sess.run([losses, 'test_acc:0', train_acc_ops])
    
    sample_size = 250
    sess.run(test_init_ops, feed_dict={'batch_size:0':sample_size})
    X_test_samples, y_test_samples, features = sess.run([inputs, labels_one_hot, outputs])

In [ ]:
print('Loss:', np.mean(val_losses))
print('Acc:', val_acc)
print('Indiv accs:', indiv_accs)

## Feature Visualization

In [ ]:
mean_features = np.mean(features, axis=0)
print(mean_features.shape)

In [ ]:
start = time.time()
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(mean_features)

print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-start))

In [ ]:
labels = np.argmax(y_test_samples[0], axis=-1)

In [ ]:
plt.scatter(tsne_results[:,0], tsne_results[:,1], c=labels, cmap=plt.cm.jet)
plt.colorbar()
plt.show()